In [7]:
import numpy as np
import pickle
from time import time
from joblib import Parallel, delayed
import sys
from src.models.mlp import MLP
from src.eval import RydbergEvaluator
from src.eval.eval_rydberg import est_density_from_z_measurements,determine_phase_1D,est_order_param_1D,phase2img,est_phase_diagram,est_order_param_1D_fourier_from_measurements,est_order_param_1D_fourier,est_order_param_1D_from_measurements
from src.data.loading.dataset_rydberg import RydbergDataset,unif_sample_on_grid
# Transformer
import argparse
from constants import *
from src.training.rydberg_trainers import RydbergConditionalTransformerTrainer
from src.models.transformer import init_conditional_transformer
from src.models.mlp import MLP


import torch
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm,trange
import os

from src.utils import plot_phase_diagram

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Here we define a base schedule -- 
base_time = 3.5
ts = np.array([0,0.2,base_time])
omegas = np.array([0, 5, 5])
deltas = np.array([-10, -10, 15])
total_time = 15 # Total adiabatic evolution time of the Bloqade simulation
# We propotionally lengthen the schedules for Omega and Delta by time_ratio = total_time/base_time
time_ratio = total_time/base_time

Now we load simulated data.

In [9]:
n_qubits = 31 # number of Rydberg atoms in the 1D lattice

dim=1 # dimension of the system
ny = 1 # since we are working in 1D, this variable is fixed to 1
nx = n_qubits # effectively, we are working on a 2D lattice of dimensions nx*ny, where nx=n_qubits and ny=1.
z2_threshold=0.7 # threshold for the Z2 order parameter to determine a state is in Z2 phase
z3_threshold = 0.6 # threshold for the Z3 order parameter to determine a state is in Z3 phase

# We load simulation data for the lattice defined above with the adiabatic evolution scheduler preset above.
folder = f'data/1D-Phase_{nx}/1D-Phase_{nx}/{float(np.round(total_time,2))}µs/'
folder

'data/1D-Phase_31/1D-Phase_31/15.0µs/'

In [10]:
# extra variables we want the conditional generative variable to condition on, except for "nx", "ny", "interaction_range".
# detuning = Delta/Omega
extra_variables = ["detuning",] 
meta_dataset = RydbergDataset(dim=dim,nx = nx, ny=ny, folder=folder,n_threads=1, 
                                         var_name='interaction_range',variables = extra_variables) 
meta_dataset.est_order_params()
meta_dataset.info["phase" ] = determine_phase_1D(meta_dataset.info["Z2"], meta_dataset.info["Z3"],z2_threshold=z2_threshold,
                                                z3_threshold=z3_threshold
                                            )

In [11]:
meta_dataset.info.size

12480

In [12]:
N = 30

In [13]:
train_idxes = np.load(f'logs/rydberg_1D_boundary/{N}/train_idxes.npy')
train_set = pickle.load(open(f'logs/rydberg_1D_boundary/{N}/train_set.pkl','rb'))

In [14]:
train_idxes

array([1380,  488, 1454, 1300,  980, 1542,  739, 1394,  349,  383, 1142,
        612,  913,  939,  431,  302,  508,  981,  817,  146,  997, 1100,
        140,  271,  597,  861,  715, 1145, 1466, 1543])

In [15]:
import random

# This will consistently give you the same ordering
train_set_keys = list(train_set.keys())
train_set_values = list(train_set.values())

# Select N random indices
selected_indices = random.sample(range(len(train_set_keys)), N)

# These will correspond to the same positions
selected_dict_items = {train_set_keys[i]: train_set[train_set_keys[i]] for i in selected_indices}
selected_array_items = [train_idxes[i] for i in selected_indices]

In [16]:
train_set = selected_dict_items
train_idxes = selected_array_items

In [18]:
def parse_args(args=[]):
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, default='logs/rydberg/debug/')
    parser.add_argument('--dim',type=int,default=1)
    parser.add_argument('--nx',type=int,default=19)
    parser.add_argument('--ny',type=int,default=1)
    parser.add_argument('--total_time',type=float,default=6)
    parser.add_argument('--tf-arch', type=str, default='transformer_l4_d128_h4')
    parser.add_argument('--train-id', type=str, default="debug")
    parser.add_argument('--reps', type=int, default=1)
    parser.add_argument('--ns', type=int, default=800, help='number of samples per hamiltonian')
    parser.add_argument('--iterations', type=int, default=50000, help="training iterations")
    parser.add_argument('--eval-every', type=int, default=100)
    parser.add_argument('--eval-samples', type=int, default=10000, help='number of generated samples for evaluation')
    parser.add_argument('--k', type=int, default=1, help='number of buckets for median of means estimation')
    parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
    parser.add_argument('--verbose', type=int, default=1, choices=[0, 1])
    parser.add_argument('--epoch-mode', type=int, default=1, choices=[0, 1])
    parser.add_argument('--condition-mode', type=int, default=0, choices=[0, 1])
    parser.add_argument('--seed', type=int, default=None)
    return parser.parse_args(args)

def get_hyperparams(**kwargs):
    hparams = argparse.Namespace(
        lr=1e-3,
        wd=0,
        bs=512,
        dropout=0.0,
        lr_scheduler=WARMUP_COSINE_SCHEDULER,
        warmup_frac=0.,
        final_lr=1e-7,
        smoothing=0.0,
        use_padding=0,
        val_frac=0.25,
        cattn=0
    )

    for k, v in kwargs.items():
        setattr(hparams, k, v)

    return hparams
args = parse_args()
hparams = get_hyperparams()

In [19]:
model_name = f'transformer_boundary_nq-{n_qubits}_iter-{args.iterations//1000}k'

In [22]:
ckpt_path = f'logs/rydberg_1D_boundary/{N}/{model_name}.pth'
transformer = torch.load(ckpt_path, map_location='cpu')

# Force the model to CPU and update its device attribute
transformer = transformer.to('cpu')
if hasattr(transformer, 'device'):
    transformer.device = torch.device('cpu')

For every point in the phase diagram, we use the conditional generative model to generate `n_gen_samples` measurements, and then determine the phase by order parameters.

The predicted order parameters & phases are saved in the DataFrame `test_df`

In [23]:
n_gen_samples = 1000
order_params = ['Z2','Z3']

**Note.** the measurement generation of the trained model has randomness -- so each run may give slightly different results

In [27]:
M = 10

plot_df = meta_dataset.info.copy()
plot_df = plot_df.loc[(plot_df['detuning'] >=-1) & (plot_df['interaction_range'] <= 2.8) & (plot_df['interaction_range'] > 1)]

test_df = plot_df.copy()
test_df[order_params] = np.nan
test_df = test_df.sample(n=M, random_state=42) 
test_df.head()

,nx,ny,interaction_range,detuning,Z2,Z3,Z4,phase
222,31.0,1.0,1.30,0.820513,NaN,NaN,0.466617,Z2
1026,31.0,1.0,2.30,1.333333,NaN,NaN,0.274985,Z3
816,31.0,1.0,1.70,0.051282,NaN,NaN,0.282824,Disordered
344,31.0,1.0,2.75,1.076923,NaN,NaN,0.300289,Disordered
979,31.0,1.0,2.70,0.435897,NaN,NaN,0.281333,Disordered


In [28]:
# To load later:
# test_df = pd.read_csv('test_results.csv', index_col=0)
# test_df = pd.read_pickle('test_results.pkl')
# with open('densities.pkl', 'rb') as f:
#     densities = pickle.load(f)

In [29]:
torch.manual_seed(0) # Set seed to ensure reproduction
torch.cuda.manual_seed(0)


densities = {}
transformer.eval()

# Here we obtained order parameters from the generative model
# The genrative model was trained using trainer.train() step above 

# M is the number of testing data points we generate with the generative model 

# We use those test datapoints to plot the phase diagram and compare it with the ground truth 


for idx in tqdm(test_df.index, "Test System"):
    key = meta_dataset.keys[idx]
    condition = torch.from_numpy(np.array([key])).float()
    gen_samples = transformer.sample_batch(cond_var=condition,batch_size=n_gen_samples,
                                               num_qubits=n_qubits)
    density = gen_samples.mean(axis=0)
    densities[idx] = density
    for order_param in order_params:
        test_df.loc[idx, order_param] = est_order_param_1D_from_measurements(gen_samples,order_param=order_param,
                                                                             nx=n_qubits)
test_df['phase'] = determine_phase_1D(test_df['Z2'].values, test_df['Z3'].values,z3_threshold=z3_threshold,z2_threshold=z2_threshold)
test_df = test_df.sort_values(['phase'])

Test System:   0%|          | 0/10 [00:00<?, ?it/s]

In [30]:
# At the end of your code cell, add:

# # Save the DataFrame
# test_df.to_csv('test_results.csv', index=True)
# # or
# test_df.to_pickle('test_results.pkl')  # Preserves data types better

# # Save the densities dictionary
# import pickle
# with open('densities.pkl', 'wb') as f:
#     pickle.dump(densities, f)


In [31]:
# # To load later:
# test_df = pd.read_csv('test_results.csv', index_col=0)
# test_df = pd.read_pickle('test_results.pkl')
# with open('densities.pkl', 'rb') as f:
#     densities = pickle.load(f)

test_df

,nx,ny,interaction_range,detuning,Z2,Z3,Z4,phase
816,31.0,1.0,1.70,0.051282,0.526948,0.370520,0.282824,Disordered
344,31.0,1.0,2.75,1.076923,0.559906,0.460888,0.300289,Disordered
979,31.0,1.0,2.70,0.435897,0.533632,0.414024,0.281333,Disordered
778,31.0,1.0,1.55,0.307692,0.568137,0.398070,0.299193,Disordered
496,31.0,1.0,1.85,0.051282,0.536849,0.379615,0.283176,Disordered
222,31.0,1.0,1.30,0.820513,0.765359,0.407069,0.466617,Z2
274,31.0,1.0,1.20,2.358974,0.953751,0.377711,0.489126,Z2
869,31.0,1.0,2.00,1.717949,0.707892,0.409349,0.286203,Z2
1026,31.0,1.0,2.30,1.333333,0.567597,0.677384,0.274985,Z3
519,31.0,1.0,1.85,3.000000,0.597142,0.707258,0.462899,Z3


Now we can compare the phase diagram predicted by our trained model against the ground truth phase diagram.

In [32]:
# fig_pred = plot_phase_diagram(test_df,title=f"Predicted Phase Diagram (Ours)",
#                               train_idxes=train_idxes,hue_order=hue_order,)

Let us exclude the training points over the phase diagram -- to obtain a test set for quantative evaluation

In [33]:
test_idxes = np.sort(plot_df.index[~np.isin(plot_df.index,train_idxes)])

In [34]:
test_idxes[:10]

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

The prediction error of our model on test points in the phase diagram

In [35]:
# Option 1: Use test_df's actual indices
pred_order_params = test_df[order_params].values
true_order_params = meta_dataset.info.loc[test_df.index][order_params].values
err_CGM = np.sqrt(np.mean((pred_order_params - true_order_params)**2))
err_CGM

np.float64(0.17084161755769994)

In [36]:
def compute_acc(pred_df,true_df,idxes=None):
    if idxes is not None:
        pred_df = pred_df.loc[idxes]
        true_df = true_df.loc[idxes]
    V = pred_df['phase'].values == true_df['phase'].values
    clf_acc = np.mean(V)
    return clf_acc

In [37]:
# meta_dataset.info.loc[common_idxes]

In [38]:
# Only use indices that exist in test_df
common_idxes = test_df.index 
print(len(common_idxes)) 
clf_acc = compute_acc(
    pred_df=test_df,
    true_df=meta_dataset.info,
    idxes=common_idxes
)
print(f"Classification accuracy: {clf_acc:.2%}")

10
Classification accuracy: 80.00%
